<a href="https://colab.research.google.com/github/Liqs-v2/octopack-refactoring-evaluation/blob/main/src/refact_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Imports

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00


In [81]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

from typing import List

## Load model

In [3]:
checkpoint = "smallcloudai/Refact-1_6B-fim"
device = "cuda" # for GPU usage or "cpu" for CPU usage

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True).to(device)

tokenizer_config.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/532 [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/smallcloudai/Refact-1_6B-fim:
- configuration_gpt_refact.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_gpt_refact.py:   0%|          | 0.00/24.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/smallcloudai/Refact-1_6B-fim:
- modeling_gpt_refact.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/3.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Load dataset

In [9]:
dataset = load_dataset("bigcode/humanevalpack", "python")

In [10]:
dataset['test'][0]

{'task_id': 'Python/0',
 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
 'declaration': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n',
 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n',
 'buggy_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = elem - elem2\n             

# Evaluate model

## Build queries for prompts

Create a query format matching the one specified in appendix N of the [octopack paper](https://arxiv.org/abs/2308.07124) for `HumanEvalFix`. In Table 16 of appendix N the authors state that "If no function start or no context is present,
that part is not added to the prompt (and the preceding newline is also removed)." I interpreted the horizontal bars in their sample query for `HumanEvalFix` as newlines, thus adding "\n\n" between the "Instruction" and "Context sections as well as the "Context" and "Function start" sections.

In [40]:
sample_query = dataset['test'][0]['instruction'] + "\n\n" + dataset['test'][0]['declaration'] + dataset['test'][0]['buggy_solution'] + "\n" + dataset['test'][0]['declaration']

In [41]:
print(sample_query)

Write a Python function `has_close_elements(numbers: List[float], threshold: float) -> bool` to solve the following problem:
Check if in given list of numbers, are any two numbers closer to each other than
given threshold.
>>> has_close_elements([1.0, 2.0, 3.0], 0.5)
False
>>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
True

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = elem - elem2
                if distance < threshold:
                    return True

    return False

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:



## Prompt model for refactoring

As the authors in the [octopack paper](https://arxiv.org/abs/2308.07124) suggest (see appendix N) I "do not optimize prompts and go with the format provided by the respective
model authors or the most intuitive format if none is provided."

Since the [Refact-1.6B model](https://huggingface.co/smallcloudai/Refact-1_6B-fim) provides a prompting template for chat use, which is suggested in the task description, we use this template.

In [45]:
prompt_template = "<empty_output>SYSTEM {system}\n" \
                  "<empty_output>USER {query}\n" \
                  "<empty_output>ASSISTANT"

In [66]:
instruction = dataset['test'][0]['instruction']
context = dataset['test'][0]['declaration'] + dataset['test'][0]['buggy_solution']
function_start = dataset['test'][0]['declaration']
query = instruction + "\n\n" + context + "\n" + function_start

In [65]:
prompt = prompt_template.format(system="You are a programming assistant",
                                query=query)

In [48]:
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_length=5000, temperature=0.2)
print("-"*80)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


--------------------------------------------------------------------------------
<empty_output>SYSTEM You are a programming assistant
<empty_output>USER Write a Python function `has_close_elements(numbers: List[float], threshold: float) -> bool` to solve the following problem:
Check if in given list of numbers, are any two numbers closer to each other than
given threshold.
>>> has_close_elements([1.0, 2.0, 3.0], 0.5)
False
>>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
True

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx!= idx2:
                distance = elem - elem2
                if distance < threshold:
                    return True

    return False

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:

<empty_output>ASSISTANT from typing import List

def has_close_

In [58]:
result = tokenizer.decode(outputs[0])

False

In [69]:
sample_solution = dataset['test'][0]['declaration']+dataset['test'][0]['canonical_solution']

In [67]:
model_solution = (result.split("ASSISTANT ")[-1]).split("<empty_output>")[0]

In [82]:
def extract_non_empty_lines_from(solution: List[str]) -> List[str]:
  solution_lines = solution.split('\n')
  return [line for line in solution_lines if line.strip() != '']

def is_model_solution_canonical_solution(model_solution, sample_solution) -> bool:
  model_solution_lines = extract_non_empty_lines_from(model_solution)
  sample_solution_lines = extract_non_empty_lines_from(sample_solution)

  if len(sample_solution_lines) != len(model_solution_lines):
    return False

  solution_lines = zip(sample_solution_lines, model_solution_lines)

  for sample_solution_line, model_solution_line in solution_lines:
    if sample_solution_line != model_solution_line:
      print("Solution mismatch detected!")
      print(f"Sample solution: {sample_solution_line}")
      print(f"Model solution: {model_solution_line}")
      return False

In [83]:
is_model_solution_canonical_solution(model_solution, sample_solution)

Solution mismatch detected!
Sample solution:             if idx != idx2:
Model solution:             if idx!= idx2:


False